# Telegram Sudoku Solver — Setup and Run Notebook

Use this notebook to:
- Set your Telegram bot token securely
- Clone the GitHub repository containing the bot
- Install all dependencies (works on Google Colab and local Jupyter)
- Run the bot (uses `colab_bot.py` on Colab when available)

Edit the GitHub URL below or pass it when prompted.



In [ ]:
# Configuration: set your GitHub repo URL here (HTTPS)
# Example: https://github.com/yourusername/sudoku-telegram-bot.git
GITHUB_URL = "https://github.com/yourusername/your-repo.git"  # <- CHANGE THIS to your repo URL

import os, sys
from getpass import getpass

print("Running in Colab:", 'google.colab' in sys.modules)

# Ask for token securely (hidden input)
BOT_TOKEN = os.environ.get('TELEGRAM_BOT_TOKEN')
if not BOT_TOKEN:
    BOT_TOKEN = getpass("Enter your TELEGRAM_BOT_TOKEN: ")

# Validate and get GitHub URL
if not GITHUB_URL or not GITHUB_URL.startswith("http") or "yourusername" in GITHUB_URL:
    # Prompt if URL is default or invalid
    entered = input("Enter your GitHub repo URL (HTTPS): ").strip()
    if entered:
        GITHUB_URL = entered
    else:
        raise ValueError("Please set a valid GitHub URL")

print(f"✅ Repository URL: {GITHUB_URL}")
print(f"✅ Bot token: {'*' * 20}...{BOT_TOKEN[-4:] if len(BOT_TOKEN) > 4 else ''}")

# Store in environment for other cells
os.environ['TELEGRAM_BOT_TOKEN'] = BOT_TOKEN
os.environ['GITHUB_URL'] = GITHUB_URL


In [ ]:
# Check if running in Colab
import sys
is_colab = 'google.colab' in sys.modules
print('Running in Colab:', is_colab)
print('Environment ready for next steps...')


In [ ]:
# Clone or update the repository
import os
import subprocess
from pathlib import Path

REPO_DIR = "repo"

# Get GITHUB_URL from environment (set in previous cell)
github_url = os.environ.get('GITHUB_URL')

if not github_url or not github_url.startswith("http"):
    github_url = input("Enter your GitHub repo URL (HTTPS): ").strip()
    os.environ['GITHUB_URL'] = github_url

if not os.path.exists(os.path.join(REPO_DIR, '.git')):
    print(f"📥 Cloning {github_url} into {REPO_DIR}...")
    subprocess.run(['git', 'clone', github_url, REPO_DIR], check=True)
    print("✅ Repository cloned successfully!")
else:
    print("📥 Repo exists. Pulling latest changes...")
    result = subprocess.run(['git', '-C', REPO_DIR, 'pull', '--ff-only'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Repository updated!")
    else:
        print("⚠️  Could not pull (may be up to date)")

# Show repository contents
print('\n📁 Repository contents:')
files = [p for p in sorted(Path(REPO_DIR).glob('**/*')) 
         if p.is_file() and '.git' not in str(p)]
for p in files:
    print(f"  • {p.relative_to(REPO_DIR)}")


In [ ]:
# Install dependencies
import os
import sys
import subprocess

REPO_DIR = "repo"

# Install Python packages
if os.path.exists(os.path.join(REPO_DIR, 'requirements.txt')):
    print("Installing from requirements.txt...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 
                   os.path.join(REPO_DIR, 'requirements.txt')], check=True)
else:
    print("Installing default packages...")
    packages = [
        'python-telegram-bot==20.7',
        'opencv-python==4.8.1.78',
        'numpy==1.24.3',
        'Pillow==10.1.0',
        'pytesseract==0.3.10'
    ]
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + packages, check=True)

# On Colab, install Tesseract OCR
if 'google.colab' in sys.modules:
    print('Installing Tesseract OCR...')
    subprocess.run(['apt-get', '-qq', 'update'], check=True)
    subprocess.run(['apt-get', '-qq', 'install', '-y', 'tesseract-ocr'], check=True)
    print('Tesseract installed!')
else:
    print('Skipping system Tesseract install (not Colab).')
    print('If needed, install manually: sudo apt-get install tesseract-ocr')

print('\n✅ All dependencies installed!')


In [ ]:
# Run the bot (blocks until you stop the cell)
import os, sys
import importlib.util

os.environ['TELEGRAM_BOT_TOKEN'] = os.environ.get('TELEGRAM_BOT_TOKEN', '')
if not os.environ['TELEGRAM_BOT_TOKEN']:
    raise RuntimeError("TELEGRAM_BOT_TOKEN is not set")

# Change working directory into repo for imports
os.chdir('repo')
print('CWD:', os.getcwd())

is_colab = 'google.colab' in sys.modules

# Prefer colab_bot.py on Colab; fall back to bot.py
if is_colab and os.path.exists('colab_bot.py'):
    from colab_bot import main
    print('Starting colab_bot.py ...')
    main()
elif os.path.exists('bot.py'):
    from bot import main
    print('Starting bot.py ...')
    main()
else:
    raise FileNotFoundError('Could not find colab_bot.py or bot.py in the repo directory')
